# Data Preprocessing

In [497]:
import pandas as pd

In [498]:
df = pd.read_csv("data/training_set.csv")

In [499]:
df["rating"] = df["Click"] #+ df["Purchase"] * 9
df = df[df["rating"] >= 0.5]

In [500]:
user_list = df.groupby('UserId')['ItemId'].nunique()
user_list_denoise = user_list[(user_list<=1000)].index.to_list()
test = pd.read_csv('data/test_set_private.csv', names=['user_id'] + [f'item_id_{i}' for i in range(1,1001)])
test_user_id = test['user_id'].values
final_user_list = []
final_user_list.extend(user_list_denoise)
final_user_list.extend(test_user_id)
user_list_denoise = list(set(final_user_list))

raw_df = df[df.UserId.isin(user_list_denoise)].reset_index().drop(columns="index")
raw_df["TrueUser"] = raw_df["UserId"]

In [501]:
def sliding_window(data: list, n=1, stride=1):
    if len(data) < n:
        yield data
    else:
        for i in range(0, len(data) - n + 1, stride):
            yield data[i:i+n]

In [502]:
window_slicing = df.groupby("UserId").agg({"ItemId": list})
window_slicing["ItemId"] = window_slicing["ItemId"].apply(lambda x : [i for i in sliding_window(x, n=50, stride=25)])
window_slicing = window_slicing.explode("ItemId").reset_index().reset_index()
window_slicing = window_slicing.rename(columns={"UserId" : "TrueUser"})
window_slicing = window_slicing.rename(columns={"index": "UserId"})
window_slicing = window_slicing.explode("ItemId")
window_slicing["rating"] = 1

In [503]:
window_slicing50 = window_slicing.copy(deep=True)

In [504]:
window_slicing = df.groupby("UserId").agg({"ItemId": list})
window_slicing["ItemId"] = window_slicing["ItemId"].apply(lambda x : [i for i in sliding_window(x, n=20, stride=10)])
window_slicing = window_slicing.explode("ItemId").reset_index().reset_index()
window_slicing = window_slicing.rename(columns={"UserId" : "TrueUser"})
window_slicing = window_slicing.rename(columns={"index": "UserId"})
window_slicing = window_slicing.explode("ItemId")
window_slicing["rating"] = 1

In [505]:
window_slicing20 = window_slicing.copy(deep=True)

In [506]:
from more_itertools import locate
sessions = []
session_id = 0
for _, i in df.groupby("UserId").agg({"ItemId": list, "Purchase": list}).reset_index().iterrows():
    userid = i["UserId"]
    for l in (list(locate(i.Purchase))):
        sessions.append((userid, session_id, i["ItemId"][l-10:l+10]))
        session_id+=1

sessions_df = pd.DataFrame(sessions, columns = ["TrueUser", "UserId", "ItemId"])
sessions_df = sessions_df.explode("ItemId")
sessions_df["rating"] = 1

In [507]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

def data_condat(*data_list):
    full = []
    for data in data_list:
        f = data.groupby("UserId").agg({"ItemId": list, "TrueUser": "max"})
        f["origin"] = get_var_name(data)
        full.append(f)


    full_df = pd.concat(full, ignore_index=True)
    # full_df = full_df.drop_duplicates(subset="ItemId")
    full_df = full_df.reset_index().rename(columns={"index": "UserId"})
    full_df = full_df.explode("ItemId")
    full_df["rating"] = 1

    return full_df

In [508]:
items_more_than_3_users = raw_df["ItemId"].value_counts()
items_more_than_3_users = items_more_than_3_users[items_more_than_3_users >= 3]
item_more_than_3_list = items_more_than_3_users.reset_index()["ItemId"].unique()
items_more_than_3_users = raw_df.merge(items_more_than_3_users, on="ItemId", how="inner")
items_more_than_3_users["TrueUser"] = items_more_than_3_users["UserId"]

In [509]:
# items_less_than_3_users = df["ItemId"].value_counts()
# items_less_than_3_users = items_less_than_3_users[items_less_than_3_users <= 10]
# items_less_than_3_list = items_less_than_3_users.reset_index()["ItemId"].unique()

# have_purchase_user = df[df["ItemId"].isin(items_less_than_3_list)].groupby("UserId").agg({"Click": "sum", "Purchase": "sum", "ItemId": list})
# have_purchase_user["Purchase_ratio"] = have_purchase_user["Purchase"] / have_purchase_user["Click"]
# have_purchase_user = have_purchase_user[have_purchase_user["Purchase_ratio"] >= 0.1]
# have_purchase_user = have_purchase_user.explode("ItemId")

# have_purchase_user = have_purchase_user.reset_index()
# # have_purchase_user = df.merge(have_purchase_user, on=["UserId", "ItemId"], how="inner")
# have_purchase_user["TrueUser"] = have_purchase_user["UserId"]

In [510]:
# # items_more_than_5_users = df["ItemId"].value_counts()
# # items_more_than_5_users = items_more_than_5_users[items_more_than_5_users >= 5]
# # item_more_than_5_list = items_more_than_5_users.reset_index()["ItemId"].unique()

# purchase_item_ratio = df.groupby("ItemId").agg({"Click": "sum", "Purchase": "sum"})
# purchase_item_ratio["Purchase_ratio"] = purchase_item_ratio["Purchase"] / purchase_item_ratio["Click"]
# purchase_item_ratio.sort_values("Purchase_ratio", ascending = False)["Purchase"]
# purchase_item_ratio = purchase_item_ratio[purchase_item_ratio["Purchase_ratio"] > 0.08]
# # purchase_item_ratio["Purchase_ratio"].quantile(0.75)
# purchase_item_ratio = df.merge(purchase_item_ratio, on="ItemId", how="inner")
# purchase_item_ratio["TrueUser"] = purchase_item_ratio["UserId"]

In [511]:
# popular_item = raw_df["ItemId"].value_counts()
# popular_item = popular_item[popular_item >= 50]
# popular_item_list = popular_item.reset_index()["ItemId"].unique()

# popular_item_with_low_purchase = df[df["ItemId"].isin(popular_item_list)].groupby("ItemId").agg({"Click": "sum", "Purchase": "sum"})
# popular_item_with_low_purchase["Purchase_ratio"] = popular_item_with_low_purchase["Purchase"] / popular_item_with_low_purchase["Click"]
# popular_item_with_low_purchase.sort_values("Purchase_ratio", ascending = False)["Purchase"]
# popular_item_with_low_purchase = popular_item_with_low_purchase[popular_item_with_low_purchase["Purchase_ratio"] >= 0]
# # purchase_item_ratio["Purchase_ratio"].quantile(0.75)
# popular_item_with_low_purchase = df.merge(popular_item_with_low_purchase, on="ItemId", how="inner")
# popular_item_with_low_purchase["TrueUser"] = popular_item_with_low_purchase["UserId"]

In [512]:
# popular_item = raw_df["ItemId"].value_counts()
# popular_item = popular_item[popular_item >= 50]
# popular_item_list = popular_item.reset_index()["ItemId"].unique()

# popular_item_with_high_purchase = df[df["ItemId"].isin(popular_item_list)].groupby("ItemId").agg({"Click": "sum", "Purchase": "sum"})
# popular_item_with_high_purchase["Purchase_ratio"] = popular_item_with_high_purchase["Purchase"] / popular_item_with_high_purchase["Click"]
# popular_item_with_high_purchase.sort_values("Purchase_ratio", ascending = False)["Purchase"]
# popular_item_with_high_purchase = popular_item_with_high_purchase[popular_item_with_high_purchase["Purchase_ratio"] > 0.5]
# # purchase_item_ratio["Purchase_ratio"].quantile(0.75)
# popular_item_with_high_purchase = df.merge(popular_item_with_high_purchase, on="ItemId", how="inner")
# popular_item_with_high_purchase["TrueUser"] = popular_item_with_high_purchase["UserId"]

In [513]:
only_purchase = raw_df[raw_df["Purchase"] == 1].reset_index().drop(columns="index")
only_purchase["TrueUser"] = only_purchase["UserId"]

# VAE Data Process

In [514]:
import os
import sys

import numpy as np
from scipy import sparse
import pandas as pd

In [515]:
vae_config = {
    "min_items_per_user": 2,
    "min_users_per_item": 1,
    "n_heldout_users": 100
}
output_dir = "runs/RecVAE"

In [516]:
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [517]:
df.to_csv(f"{output_dir}/data.csv", index=False)

In [518]:
df = data_condat(raw_df, window_slicing20, window_slicing50, sessions_df, items_more_than_3_users, only_purchase, 
                 # purchase_item_ratio, popular_item_with_low_purchase, popular_item_with_high_purchase, have_purchase_user,
                )
# df = data_condat(window_slicing50)

In [519]:
df

,UserId,ItemId,TrueUser,origin,rating
0,0,WCJ3v3bGfi,00CqJFh8ey,raw_df,1
0,0,ERHK6OW0BA,00CqJFh8ey,raw_df,1
0,0,v5DrRkI5jd,00CqJFh8ey,raw_df,1
0,0,7o8G2oZDME,00CqJFh8ey,raw_df,1
0,0,d6qTUg1iUa,00CqJFh8ey,raw_df,1
...,...,...,...,...,...
219217,219217,kJucNmxdaK,zyfXpoPInl,popular_item_with_low_purchase,1
219218,219218,cQ23hskQW7,zywXEdYD2i,popular_item_with_low_purchase,1
219219,219219,0JigBxUIcZ,zz6FlSEvck,popular_item_with_low_purchase,1
219219,219219,hYPuswhmpJ,zz6FlSEvck,popular_item_with_low_purchase,1


In [520]:
df.to_csv("multiverse_data.csv", index=False)

In [521]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

def filter_triplets(tp, min_uc, min_sc): 
    if min_sc > 0:
        itemcount = get_count(tp, 'ItemId')
        tp = tp[tp['ItemId'].isin(
            itemcount[itemcount["size"] >= min_sc]["ItemId"].unique()
        )]
    
    if min_uc > 0:
        usercount = get_count(tp, 'UserId')
        tp = tp[tp['UserId'].isin(
            usercount[usercount["size"] >= min_uc]["UserId"].unique()
        )]
    
    usercount, itemcount = get_count(tp, 'UserId'), get_count(tp, 'ItemId') 
    return tp, usercount, itemcount


In [522]:
raw_data, user_activity, item_popularity = filter_triplets(df, vae_config["min_items_per_user"], vae_config["min_users_per_item"])

sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

# Get the unique user IDs
unique_uid = user_activity["UserId"].unique()

# Shuffle the user IDs to create a random permutation
np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

# Get the total number of users
n_users = unique_uid.size

n_users

After filtering, there are 2013679 watching events from 199395 users and 83102 movies (sparsity: 0.012%)


199395

## Sampling Train Test Valid

In [523]:
unique_trueuser = raw_data["TrueUser"].unique()
n_trueuser = len(unique_trueuser)

In [524]:
# Split the users into three sets: training, validation, and testing
# tr_users = np.concat([unique_uid[:(n_users - vae_config["n_heldout_users"] * 2)], test["user_id"].unique()])

# tr_users = unique_uid[:(n_users - vae_config["n_heldout_users"] * 2)]
# vd_users = unique_uid[(n_users - vae_config["n_heldout_users"] * 2): (n_users - vae_config["n_heldout_users"])]
# te_users = unique_uid[(n_users - vae_config["n_heldout_users"]):]

tr_users = unique_trueuser[:(n_trueuser - vae_config["n_heldout_users"])]
vd_users = unique_trueuser[(n_trueuser - vae_config["n_heldout_users"]):]
te_users = unique_trueuser[(n_trueuser - vae_config["n_heldout_users"]):]

# Get the training data (i.e., the plays made by the training users)
train_plays = raw_data.loc[raw_data['TrueUser'].isin(tr_users)]

# Get the unique movie IDs in the training data
unique_sid = pd.unique(train_plays['ItemId'])

# Create mapping id to index
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [525]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(os.path.join(output_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)
        
with open(os.path.join(output_dir, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)

In [526]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('UserId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [527]:
vad_plays = raw_data.loc[raw_data['TrueUser'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['ItemId'].isin(unique_sid)]

vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

test_plays = raw_data.loc[raw_data['TrueUser'].isin(te_users)]
test_plays = test_plays.loc[test_plays['ItemId'].isin(unique_sid)]

test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
1000 users sampled
0 users sampled
1000 users sampled


In [528]:
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['UserId']))
    sid = list(map(lambda x: show2id[x], tp['ItemId']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [529]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(output_dir, 'train.csv'), index=False)

vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(output_dir, 'validation_tr.csv'), index=False)

vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(output_dir, 'validation_te.csv'), index=False)

test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(output_dir, 'test_tr.csv'), index=False)

test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(output_dir, 'test_te.csv'), index=False)

In [530]:
n_items = len(unique_sid)
n_users = len(unique_uid)

In [531]:
n_items

82803

# Train VAE 

In [ ]:
# !python recvae/run.py --dataset outputdir−−n−epochs50−−gamma0.005−−savemodeloutput_dir --n-epochs 50 --gamma 0.005 --save_model output_dir/model.pt

In [ ]:
!python recvae/run.py --dataset $output_dir --n-epochs 100 --hidden-dim 3072 --latent-dim 2048 --gamma 0.01 --implicitslim --step 15 --lambd 500 --alpha 1 --lr 1e-4 --batch-size 512 --threshold 1000 --save_model $output_dir/model.pt